In [149]:
import numpy as np
import pandas as pd
from functools import reduce
import time
import sys
import os   
import re
import importlib
import math
import sklearn
from sklearn import preprocessing
from scipy.spatial import Delaunay
from scipy import stats
from matplotlib import cm
import matplotlib.animation as animation
import matplotlib.colors as colors
import matplotlib.pyplot as plt
%matplotlib widget

## DO THIS TO RUN R CODE WITHIN PYTHON NOTEBOOK
# %pip install r2py
# os.environ['R_HOME'] = "/usr/local/Cellar/r/4.0.4/lib/R"
# import rpy2.robjects as robjects

## Definitions

## R code
install.packages("alphashape3d")
install.packages("tidyverse")
library("alphashape3d")
library("tidyverse")

getCriticalAlpha <- function(components, alphas, floor=10){
  criticalIndex <- 1
  for (i in 1:length(components)){
    comp = as.numeric(components[[i]])
    if (any(comp != 1)){
      criticalIndex <- i
    }
  }
  criticalIndex <- criticalIndex + 1
  print(alphas[[criticalIndex]])
  return(list(criticalIndex, alphas[[criticalIndex]]))
}

rdf <- read_csv("/Users/phil/Downloads/all_ex.csv") %>% rename(index = X1)
rdf_s <- rdf %>% filter(species=="s")
rdf_d <- rdf %>% filter(species=="d")

logAlphas <- exp(seq(log(0.1), log(10000), length.out = 200))
alphaShapes_s <- rdf_s %>% select(Rx, Ry, cRz) %>% as.matrix() %>% ashape3d(logAlphas)
alphaShapes_d <- rdf_d %>% select(Rx, Ry, cRz) %>% as.matrix() %>% ashape3d(logAlphas)

comp_s <- components_ashape3d(alphaShapes_s, indexAlpha = "all")
comp_d <- components_ashape3d(alphaShapes_d, indexAlpha = "all")

crit_s <- getCriticalAlpha(comp_s, logAlphas)
crit_d <- getCriticalAlpha(comp_d, logAlphas)

search_df_s <- tibble(alpha = logAlphas, components = comp_s, alphaIndex = seq(1,length(logAlphas))) 
search_df_d <- tibble(alpha = logAlphas, components = comp_d, alphaIndex = seq(1,length(logAlphas)))

volumes_s <- volume_ashape3d(alphaShapes_s, byComponents = FALSE, indexAlpha = as.vector(search_df_s['alphaIndex'][[1]]))
volumes_d <- volume_ashape3d(alphaShapes_d, byComponents = FALSE, indexAlpha =  as.vector(search_df_d['alphaIndex'][[1]]))
# normals_s <- surfaceNormals(alphaShapes_s, indexAlpha="all")
# normals_d <- surfaceNormals(alphaShapes_d, indexAlpha="all")







### Function Declarations

In [150]:
def format3dPlot(ax, title, xRange, yRange, zRange, view=None, mode='rotations', color='grey'):
    ax.set_title(title)
    if view:
        ax.view_init(view[0], view[1])
    if mode == 'rotations':
        ax.set_xlabel('- sup   X   + prn', size='small', color=colors['red'])
        ax.set_ylabel('- dep   Y   + ele', size='small', color=colors['green'])
        ax.set_zlabel('- ret   Z   + prt', size='small', color=colors['blue'])
    elif mode == 'pca':
        ax.set_xlabel('PC1', size='small')
        ax.set_ylabel('PC2', size='small')
        ax.set_zlabel('PC3', size='small')
    ax.set_xlim(xRange[0], xRange[1])
    ax.set_ylim(yRange[0], yRange[1])
    ax.set_zlim(zRange[0], zRange[1])  
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    
    ax.w_xaxis.line.set_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_yaxis.line.set_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))
    ax.minorticks_off()
    ax.tick_params(reset=True,colors=color, labelsize='x-small')
    return ax

def on_move(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
    else:
        return
    fig.canvas.draw_idle()
    

def on_move4(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
        ax2.view_init(elev=ax0.elev, azim=ax0.azim)
        ax3.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
        ax2.view_init(elev=ax1.elev, azim=ax1.azim)
        ax3.view_init(elev=ax1.elev, azim=ax1.azim)
    elif event.inaxes == ax2:
        ax0.view_init(elev=ax2.elev, azim=ax2.azim)
        ax1.view_init(elev=ax2.elev, azim=ax2.azim)
        ax3.view_init(elev=ax2.elev, azim=ax2.azim)
    elif event.inaxes == ax3:
        ax0.view_init(elev=ax3.elev, azim=ax3.azim)
        ax1.view_init(elev=ax3.elev, azim=ax3.azim)
        ax2.view_init(elev=ax3.elev, azim=ax3.azim)
    else:
        return
    fig.canvas.draw_idle()
    
def on_move6(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
        ax2.view_init(elev=ax0.elev, azim=ax0.azim)
        ax3.view_init(elev=ax0.elev, azim=ax0.azim)
        ax4.view_init(elev=ax0.elev, azim=ax0.azim)
        ax5.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
        ax2.view_init(elev=ax1.elev, azim=ax1.azim)
        ax3.view_init(elev=ax1.elev, azim=ax1.azim)
        ax4.view_init(elev=ax1.elev, azim=ax1.azim)
        ax5.view_init(elev=ax1.elev, azim=ax1.azim)
    elif event.inaxes == ax2:
        ax0.view_init(elev=ax2.elev, azim=ax2.azim)
        ax1.view_init(elev=ax2.elev, azim=ax2.azim)
        ax3.view_init(elev=ax2.elev, azim=ax2.azim)
        ax4.view_init(elev=ax2.elev, azim=ax2.azim)
        ax5.view_init(elev=ax2.elev, azim=ax2.azim)
    elif event.inaxes == ax3:
        ax0.view_init(elev=ax3.elev, azim=ax3.azim)
        ax1.view_init(elev=ax3.elev, azim=ax3.azim)
        ax2.view_init(elev=ax3.elev, azim=ax3.azim)
        ax4.view_init(elev=ax3.elev, azim=ax3.azim)
        ax5.view_init(elev=ax3.elev, azim=ax3.azim)
    elif event.inaxes == ax4:
        ax0.view_init(elev=ax4.elev, azim=ax4.azim)
        ax1.view_init(elev=ax4.elev, azim=ax4.azim)
        ax2.view_init(elev=ax4.elev, azim=ax4.azim)
        ax3.view_init(elev=ax4.elev, azim=ax4.azim)
        ax5.view_init(elev=ax4.elev, azim=ax4.azim)
    elif event.inaxes == ax5:
        ax0.view_init(elev=ax5.elev, azim=ax5.azim)
        ax1.view_init(elev=ax5.elev, azim=ax5.azim)
        ax2.view_init(elev=ax5.elev, azim=ax5.azim)
        ax3.view_init(elev=ax5.elev, azim=ax5.azim)
        ax4.view_init(elev=ax5.elev, azim=ax5.azim)
    else:
        return
    fig.canvas.draw_idle()
    
# def addCosGrid(gridAx, xRange, yRange, zRange, interval, xLevels=1, alpha=0, **kwargs):  
#     xMin = math.floor(xRange[0])
#     xMax = math.ceil(xRange[1])+1
#     yMin = math.floor(yRange[0])
#     yMax = math.ceil(yRange[1])+1
#     zMin = math.floor(zRange[0])
#     zMax = math.ceil(zRange[1])+1
#     xs= np.arange(xMin, xMax+1, 1)
#     ys= np.arange(yMin, yMax, 1)
#     xSize = abs(xMin)+abs(xMax)
#     ySize = abs(yMin)+abs(yMax)
#     zSize = abs(zMin)+abs(zMax)
#     alphas = np.ones(xSize)*alpha
#     xx, yy = np.meshgrid(xs, ys)
#     cxx = xx*np.cos(np.radians(yy))
#     zMaxMinMax = max((abs(zMin),abs(zMax)))
#     if xLevels <2:
#         zs = np.zeros(zSize)
#     else:
#         zs = np.linspace(zMaxMinMax*-1, zMaxMinMax, xLevels)
#     for xLevel in list(range(xLevels)):
#         zz = np.ones((cxx.shape[0],cxx.shape[1]))*zs[xLevel]
#         gridAx.plot_wireframe(cxx, yy, zz, rcount = xSize/interval, ccount=ySize/interval, **kwargs) 

def addCosGrid(gridAx, xRange, yRange, zRange, interval, xLevels=1, alpha=0, **kwargs):  
    xMin = math.floor(xRange[0])
    xMax = math.ceil(xRange[1])+1
    yMin = math.floor(yRange[0])
    yMax = math.ceil(yRange[1])+1
    zMin = math.floor(zRange[0])
    zMax = math.ceil(zRange[1])+1
    zs= np.arange(zMin, zMax+1, 1)
    ys= np.arange(yMin, yMax, 1)
    xSize = abs(xMin)+abs(xMax)
    ySize = abs(yMin)+abs(yMax)
    zSize = abs(zMin)+abs(zMax)
    alphas = np.ones(zSize)*alpha
    zz, yy = np.meshgrid(zs, ys)
    czz = zz*np.cos(np.radians(yy))
    xMaxMinMax = max((abs(xMin),abs(xMax)))
    if xLevels <2:
        xs = np.zeros(xSize)
    else:
        xs = np.linspace(xMaxMinMax*-1, xMaxMinMax, xLevels)
    for xLevel in list(range(xLevels)):
        xx = np.ones((czz.shape[0],czz.shape[1]))*xs[xLevel]
        gridAx.plot_wireframe(xx, yy, czz, rcount = zSize/interval, ccount=ySize/interval, **kwargs) 



## Imports
### Import model data

In [327]:
df_M = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/Opossum_Tegu_ROMs_transformed.csv',index_col=0)
## flip values: in rob's CS, -X = pronation and +X = supination
df_M.columns = ['Rx', 'Ry', 'Rz', 'species', 'cRz']
df_M['radRy'] = np.radians(df_M['Ry'])
df_M['cosRy'] = np.cos(df_M['radRy'])
df_M['RzcosRy'] = df_M['cosRy'].multiply(df_M['Rz'])


df_M['philsway'] = df_M['RzcosRy']

Acentral = (df_M_S['Rz'].max() + df_M_S['Rz'].min())/2
df_M['robsway_adduniform'] = (df_M['Rz']-Acentral).multiply(df_M['cosRy'])+Acentral
df_M['robsway_noadduniform'] = (df_M['Rz']-Acentral).multiply(df_M['cosRy'])


df_M['Rx'] *= -1
df_M.loc[df_M['species']=='Opossum','Rx'] += 50
df_M.loc[df_M['species']=='Opossum','Ry'] -= 8
df_M.loc[df_M['species']=='Opossum','Rz'] += 2


df_M_S = df_M.loc[df_M['species']=='Tegu'].copy()
df_M_D = df_M.loc[df_M['species']=='Opossum'].copy()




Acentral_S = (df_M_S['Rz'].max() + df_M_S['Rz'].min())/2
df_M_S['robsway'] = (df_M_S['Rz']-Acentral_S).multiply(df_M_S['cosRy'])+Acentral_S
df_M_S['robsway_noadd'] = (df_M_S['Rz']-Acentral_S).multiply(df_M_S['cosRy'])
df_M_S['robsway_fromzero'] = (df_M_S['Rz']).multiply(df_M_S['cosRy'])

Acentral_D = (df_M_D['Rz'].max() + df_M_D['Rz'].min())/2
df_M_D['robsway'] = (df_M_D['Rz']-Acentral_D).multiply(df_M_D['cosRy'])+Acentral_D
df_M_D['robsway_noadd'] = (df_M_D['Rz']-Acentral_D).multiply(df_M_D['cosRy'])
df_M_D['robsway_fromzero'] = (df_M_D['Rz']).multiply(df_M_D['cosRy'])


In [321]:
df_M.loc[df_M['species']=='Opossum']

,Rx,Ry,Rz,species,cRz,radRy,cosRy,RzcosRy,philsway,robsway_adduniform,robsway_noadduniform
2538,45,-40,-80,Opossum,-72.981333,-0.698132,0.766044,-61.283555,-61.283555,-65.962667,-45.962667
2539,135,-10,-170,Opossum,-168.176930,-0.174533,0.984808,-167.417318,-167.417318,-167.721163,-147.721163
2540,135,-50,-140,Opossum,-107.850885,-0.872665,0.642788,-89.990265,-89.990265,-97.134513,-77.134513
2541,135,-50,-130,Opossum,-101.423009,-0.872665,0.642788,-83.562389,-83.562389,-90.706637,-70.706637
2542,135,-50,-120,Opossum,-94.995133,-0.872665,0.642788,-77.134513,-77.134513,-84.278761,-64.278761
...,...,...,...,...,...,...,...,...,...,...,...
5849,-45,-130,-130,Opossum,1.423009,-2.268928,-0.642788,83.562389,83.562389,50.706637,70.706637
5850,-45,-130,-120,Opossum,-5.004867,-2.268928,-0.642788,77.134513,77.134513,44.278761,64.278761
5851,-45,-130,-110,Opossum,-11.432743,-2.268928,-0.642788,70.706637,70.706637,37.850885,57.850885
5852,-45,-130,-100,Opossum,-17.860620,-2.268928,-0.642788,64.278761,64.278761,31.423009,51.423009


In [313]:
df_Mtest = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/Opossum_Tegu_ROMs_transformed.csv',index_col=0)
df_Mtest.loc[df_M['species']=='Opossum']

,X,Y,Z,Species,cZ
2538,0,-40,-80,Opossum,-72.981333
2539,-90,-10,-170,Opossum,-168.176930
2540,-90,-50,-140,Opossum,-107.850885
2541,-90,-50,-130,Opossum,-101.423009
2542,-90,-50,-120,Opossum,-94.995133
...,...,...,...,...,...
5849,90,-130,-130,Opossum,1.423009
5850,90,-130,-120,Opossum,-5.004867
5851,90,-130,-110,Opossum,-11.432743
5852,90,-130,-100,Opossum,-17.860620


### Import JCS data

In [208]:
jcs85R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_right_triceps_lat-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs88L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_left_teresmaj_pec-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs88R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_trilong_pec-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs92R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv92_biceps_3pec_delt-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs101L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101_11Apr-Phil-2020-04-13-diff/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs105R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm105-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs126 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm126-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs126L = jcs126.filter(like='Ldata', axis=1)
jcs126R = jcs126.filter(like='Rdata', axis=1)   
jcs85L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_left_biceps_teres_lat-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scaphum', axis=1)
jcs101R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101right_biceps_triceps-Phil-2020-06-08/maya/JCS.csv', 
                     index_col=0).filter(like='scap_hum', axis=1)
jcs108 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm108-Phil-2020-06-09/maya/JCS.csv', 
                     index_col=0).filter(like='scaphum', axis=1)
jcs108L = jcs108.filter(like='ldata', axis=1)
jcs108R = jcs108.filter(like='rdata', axis=1)

jcs_dfs = {'jcs_85_R':jcs85R, 'jcs_85_L':jcs85L, 'jcs_88_L':jcs88L, 'jcs_88_R':jcs88R, 'jcs_92_R':jcs92R, 'jcs_101_L':jcs101L, 'jcs_101_R':jcs101R, 'jcs_105_R':jcs105R, 'jcs_108_L':jcs108L, 'jcs_108_R':jcs108R, 'jcs_126_L':jcs126L, 'jcs_126_R':jcs126R}
jcs_cols = ['Tx','Ty','Tz','Rx','Ry','Rz']
species_dict = {'85':'d','88':'d','92':'d','101':'d','105':'s','108':'s','126':'s'}
humeral_lengths = {'85_R':56.750, '85_L':56.620, '88_R':54.015, '88_L':53.454, '92_R':55.358, '101_R':59.293,'101_L':60.039,'105_R':46.588,'108_R':42.134,'108_L':42.824,'126_R':43.666,'126_L':43.296}

colors = {'red':'#B74B4B', 'green':'#8EC15A', 'blue':'#5083D2'}
orangeYellowSeafoam = ['#54100F','#9C4622','#DC8528','#FFD793','#BCE1D0','#3CA9A6','#08676A','#012C2D']
redYellowBlue = ['#451021','#A62547','#E47961','#F8D4AC','#BDE2CF','#559FBA','#1C5796','#012C2D']
teguColor = orangeYellowSeafoam[5]
opossumColor = orangeYellowSeafoam[3]
plt.rcParams.update({
    'grid.linewidth':  0,
    'grid.color':  'lightgrey',
    'savefig.facecolor': (0.0, 0.0, 0.0, 0.0), 
    'savefig.transparent': True,
})




def hexToRgb(hexStr):
    decR = int(hexStr[1:3],16)
    decG = int(hexStr[3:5],16)
    decB = int(hexStr[5:7],16)
    decTup = (decR, decG, decB)
    return decTup

def renameAndDrop(df, colnames):
    df.columns = colnames   
    df1 = df.loc[((df['Tx']!=0)&(df['Ty']!=0)&(df['Tz']!=0)&(df['Rx']!=0)&(df['Ry']!=0)&(df['Rz']!=0))]
    return df1

jcs_dfs = {key:renameAndDrop(value, jcs_cols) for (key,value) in jcs_dfs.items()}

### Import MMA data

In [209]:
ma85R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_right_triceps_lat-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma88L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_left_teresmaj_pec-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma88R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_trilong_pec-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma92R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv92_biceps_3pec_delt-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma101L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101_11Apr-Phil-2020-04-13-diff/maya/momentArms.csv', 
                     index_col=0)
ma105R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm105-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma126 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm126-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma126L = ma126.filter(like='L_jnt', axis=1)
ma126R = ma126.filter(like='R_jnt', axis=1)
ma85L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_left_biceps_teres_lat-Phil-2020-06-09/maya/MomentArms.csv', 
                     index_col=0)
ma101R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101right_biceps_triceps-Phil-2020-06-08/maya/MA.csv', 
                     index_col=0)
ma108 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm108-Phil-2020-06-09/maya/MomentArms.csv', 
                     index_col=0)
ma108L = ma108.filter(like='l_jnt', axis=1)
ma108R = ma108.filter(like='r_jnt', axis=1)

ma_dfs = {'ma_85_R':ma85R, 'ma_85_L':ma85L, 'ma_88_L':ma88L, 'ma_88_R':ma88R, 'ma_92_R':ma92R, 'ma_101_L':ma101L, 'ma_101_R':ma101R, 'ma_105_R':ma105R, 'ma_108_L':ma108L, 'ma_108_R':ma108R, 'ma_126_L':ma126L, 'ma_126_R':ma126R}

def filterMaByJcs(maDict, jcsDict):
    newDict = maDict.copy() 
    for (key, value) in maDict.items():
        newDict[key] = newDict[key][newDict[key].index.isin(jcsDict['jcs'+'_'+key.split('_',1)[-1]].index)]
    return newDict

ma_dfs = filterMaByJcs(ma_dfs, jcs_dfs)

### Process raw data

In [210]:
aliases = { 'pec_crn':r'(?=.*pec)(?=.*cr).*',
            'pec_int':r'(?=.*pec)(?=.*int).*',
            'pec_cdl':r'(?=.*pec)(?=.*cau|.*c.*d.*l).*',
            'lat_crn':r'(?=.*lat)(?!.*cau|.*c.*d.*l).*',
            'lat_cdl':r'(?=.*lat)(?=.*cau|.*c.*d.*l).*',
            'dac':r'(.*dac|.*acr).*',
            'dsc':r'(.*dsc|.*sc.*p).*',
            'bic':r'(.*bic).*',
            'trl':r'(.*tri).*', 
            'tmj':r'(.*tmj|.*teres).*' }

def getMuscleNames(maDict):
    muscle_names = [set(value.columns) for (key,value) in maDict.items()]
    muscle_names = set().union(*muscle_names)
    muscle_names = {name for name in muscle_names}
    return muscle_names

def getAliases(aliasDict, muscleNamesList):
    result = {}
    for (key,value) in aliasDict.items():
        matches = [re.match(value, name, re.IGNORECASE) for name in muscleNamesList]
        matched_names = [name.group(0) for name in matches if name]
        result[key] = matched_names
    all_matches = {item for sublist in result.values() for item in sublist}
    missedA = all_matches.difference(muscleNamesList)
    missedB = muscleNamesList.difference(all_matches)
    missed = missedA.union(missedB)
    if missed:
        print('Failed to match some muscle names; edit regexes and try again: ' +str(missed))
    return result

def reverseAliasMap(mappedAliasesDict):
    result = {}
    for (key, value) in mappedAliasesDict.items():
        for subvalue in value:
            result[subvalue]=key+'_'+subvalue.split('.')[-1]
    return result

def renameMusclesWithMap(maDict, nameMap):
    newDict = maDict.copy() 
    for (key, value) in newDict.items():
        newDict[key] = newDict[key].rename(columns=nameMap)
    return newDict

def generateNewCols(inputDict):
    newDict = inputDict.copy() 
    for (key, value) in newDict.items():
        newDict[key]['frame'] = newDict[key].index
        newDict[key]['animal'] = key.split('_')[1]
        newDict[key]['side'] = key.split('_')[-1]
        newDict[key]['species'] = species_dict[key.split('_')[1]]
        newDict[key]['humeralLengthMm'] = humeral_lengths[key.split('_',1)[1]]
        newDict[key]['UID'] = newDict[key]['species']+newDict[key]['animal']+'f'+newDict[key]['frame'].astype(str)
        newDict[key] = newDict[key].set_index('UID')
    return newDict


         
raw_muscle_names = getMuscleNames(ma_dfs)
alias_map = getAliases(aliases, raw_muscle_names)
reverse_alias_map = reverseAliasMap(alias_map)
ma_dfs = renameMusclesWithMap(ma_dfs, reverse_alias_map)
ma_dfs = generateNewCols(ma_dfs)
jcs_dfs = generateNewCols(jcs_dfs)


#merge jcs and moment arm data
jcs_concat = pd.concat(jcs_dfs.values())
ma_concat = pd.concat(ma_dfs.values())
all_data = pd.merge(jcs_concat, ma_concat)

#scale moment arms and translations
re_t = r'(T[xyz])'
t_columns = [col for col in all_data.columns if re.match(re_t, col)]
ma_columns = [col for col in all_data.columns if col[-2:] == 'ma']
t_ma_all_data = all_data[t_columns+ma_columns]
scaled_t_ma_all_data = t_ma_all_data.divide(all_data['humeralLengthMm'],axis=0)
scaled_t_ma_all_data = scaled_t_ma_all_data.add_prefix('scaled_')

#cosine-correct rotations
jcs_alldata = all_data.loc[:,['Rx','Ry','Rz']]
##old wrong
# jcs_alldata['radRy'] = np.radians(jcs_alldata['Ry'])
# jcs_alldata['cosRy'] = np.cos(jcs_alldata['radRy'])
# jcs_alldata['RxcosRy'] = jcs_alldata['cosRy'].multiply(jcs_alldata['Rx'])
# all_data['cRx'] = jcs_alldata['RxcosRy']
jcs_alldata['radRy'] = np.radians(jcs_alldata['Ry'])
jcs_alldata['cosRy'] = np.cos(jcs_alldata['radRy'])
jcs_alldata['RzcosRy'] = jcs_alldata['cosRy'].multiply(jcs_alldata['Rz'])

all_data['cRz'] = jcs_alldata['RzcosRy']



df = all_data.join(scaled_t_ma_all_data)
df= df[df['Ry']<90]
# col_order = ['Tx', 'Ty', 'Tz', 'Rx', 'Ry', 'Rz', 'cRz', frame', 'animal', 'side','species', 'humeralLengthMm']+sorted(t_columns)+sorted(ma_columns)
# df = df[col_order]

#randomly undersample opossum data to match tegu n
# numerical_cols = df.select_dtypes(include='float').reset_index(level=0, inplace=False)
# numerical_cols_nona = numerical_cols.fillna(0)
# rus = RandomUnderSampler(sampling_strategy=1)
# X_resampled, y_resampled = rus.fit_resample(numerical_cols_nona, df['species'])
# df = df.iloc[X_resampled['index']].sort_index()

dfS = df[df['species']=='s']
dfD = df[df['species']=='d']


<ipython-input-210-35b505cdd2b8>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDict[key]['frame'] = newDict[key].index
<ipython-input-210-35b505cdd2b8>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newDict[key]['animal'] = key.split('_')[1]
<ipython-input-210-35b505cdd2b8>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

### Get experimental data ranges

In [211]:
#calculate ranges
def getRotationRanges(df1, df2):
    maxRx = math.ceil(max([max(df1.Rx),max(df2.Rx)])/10)*10
    maxcRz = math.ceil(max([max(df1.cRz),max(df2.cRz)])/10)*10
    minRx = math.floor(min([min(df1.Rx),min(df2.Rx)])/10)*10
    mincRz = math.floor(min([min(df1.cRz),min(df2.cRz)])/10)*10
    maxRy = math.ceil(max([max(df1.Ry),max(df2.Ry)])/10)*10
    minRy = math.floor(min([min(df1.Ry),min(df2.Ry)])/10)*10
    maxRz = math.ceil(max([max(df1.Rz),max(df2.Rz)])/10)*10
    minRz = math.floor(min([min(df1.Rz),min(df2.Rz)])/10)*10
    result = {'maxRx':maxRx, 'maxcRz':maxcRz, 'minRx':minRx, 'mincRz':mincRz, 'maxRy':maxRy, 'minRy':minRy, 'maxRz':maxRz, 'minRz':minRz}
    return result

rotations = getRotationRanges(dfS, dfD)
maxRx, maxcRz, minRx, mincRz, maxRy, minRy, maxRz, minRz = rotations.values()


#find min, max frames
sMaxRx = dfS[dfS['Rx']==max(dfS['Rx'])]
sMinRx = dfS[dfS['Rx']==min(dfS['Rx'])]
sMaxRy = dfS[dfS['Ry']==max(dfS['Ry'])]
sMinRy = dfS[dfS['Ry']==min(dfS['Ry'])]
sMaxcRz = dfS[dfS['cRz']==max(dfS['cRz'])]
sMincRz = dfS[dfS['cRz']==min(dfS['cRz'])]

dMaxRx = dfD[dfD['Rx']==max(dfD['Rx'])]
dMinRx = dfD[dfD['Rx']==min(dfD['Rx'])]
dMaxRy = dfD[dfD['Ry']==max(dfD['Ry'])]
dMinRy = dfD[dfD['Ry']==min(dfD['Ry'])]
dMaxcRz = dfD[dfD['cRz']==max(dfD['cRz'])]
dMincRz = dfD[dfD['cRz']==min(dfD['cRz'])]

## Plots
### Experimental ROM

In [330]:
viewYZ = (0,0)
viewXZ = (0,90)
viewXY = (90,90)
viewYX = (90,0)

view3Q = (45,45)
view3Qst = (-45,45)
view3Qsw = (135,45)

startView = viewXZ

vX, vY = viewXY

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'


fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu', (minRx, maxRx), (minRy, maxRy), (mincRz, maxcRz), view=startView)
format3dPlot(ax1, 'Opossum', (minRx, maxRx), (minRy, maxRy), (mincRz, maxcRz), view=startView)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.Rx,dfS.Ry,dfS.cRz, s=3, c='#3CA9A6', depthshade=True)
ax1.scatter(dfD.Rx,dfD.Ry,dfD.cRz, s=3, c='#FFCA4E', depthshade=True)

ax0.scatter(df_M_S.Rx,df_M_S.Ry,df_M_S.philsway, s=1, c='k', depthshade=True)
ax1.scatter(df_M_D.Rx,df_M_D.Ry,df_M_D.philsway, s=1, c='k', depthshade=True)


connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### R shit here

In [658]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.conversion import localconverter
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.robjects.lib.dplyr import DataFrame
from rpy2.robjects import rl

alphashape3d = importr('alphashape3d')
tidyverse = importr('tidyverse')
robjects.r('''
        getCriticalAlpha <- function(components, alphas){
          criticalIndex <- 1
          for (i in 1:length(components)){
            comp = as.numeric(components[[i]])
            if (any(comp != 1)){
              criticalIndex <- i
            }
          }
          criticalIndex <- criticalIndex + 1
          print(alphas[[criticalIndex]])
          return(list(criticalIndex, alphas[[criticalIndex]]))
        }
        ''')
getCriticalAlpha = robjects.r['getCriticalAlpha']
with localconverter(ro.default_converter + pandas2ri.converter):
    rdfS = robjects.r['as.matrix'](ro.conversion.py2rpy(dfS[['Rx','Ry','cRz']].copy()))
    rdfD = robjects.r['as.matrix'](ro.conversion.py2rpy(dfD[['Rx','Ry','cRz']].copy()))
subTenAlphas = np.linspace(0.1,9.9, 99)
logAlphas = np.exp(np.linspace(math.log(10), math.log(100), 101))
alphas = np.concatenate((subTenAlphas,logAlphas))


def getAlphaObjects(rdf, alphas):
    alphaCols = [str(round(alpha,3)).replace('.','_') for alpha in alphas]
    print("...calculating alphas...")
    alphaShapes = alphashape3d.ashape3d(rdf, alphas)
    print("...calculating components...")
    components = alphashape3d.components_ashape3d(alphaShapes, indexAlpha="all")
    print("...calculating volumes...")
    volumes = alphashape3d.volume_ashape3d(alphaShapes, indexAlpha="all")
    print("...calculating normals...")
    normals = alphashape3d.surfaceNormals(alphaShapes, indexAlpha="all")
    print("...calculating critical alpha...")
    [crit_index, crit_val] = getCriticalAlpha(components, alphas)
    shapes = np.array(alphaShapes, dtype=object)
    tetras_df = pd.DataFrame(alphaShapes[0]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'index4',4:'intervals'},**dict(zip(range(5,len(alphas)+5),alphaCols))})
    triangles_df = pd.DataFrame(alphaShapes[1]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'on_convex_hull',4:'attached',5:'intervals1',6:'intervals2',7:'intervals3'},**dict(zip(range(8,len(alphas)+8),alphaCols))})
    edges_df = pd.DataFrame(alphaShapes[2]).rename(columns={**{0:'index1',1:'index2',2:'on_convex_hull',3:'attached',4:'intervals1',5:'intervals2',6:'intervals3'},**dict(zip(range(7,len(alphas)+7),alphaCols))})
    vertices_df = pd.DataFrame(alphaShapes[3]).rename(columns={**{0:'index',1:'on_convex_hull',2:'intervals1',3:'intervals2'},**dict(zip(range(4,len(alphas)+4),alphaCols))})
    vertices_df['x'],vertices_df['y'],vertices_df['z'] = alphaShapes[4].T[0],alphaShapes[4].T[1],alphaShapes[4].T[2]
    components_df = pd.DataFrame(np.array(components, dtype=object).T).rename(columns=dict(zip(range(0,len(alphas)+1),alphaCols)))
    volumes_df = pd.DataFrame(np.array(volumes, dtype=object)).transpose().rename(columns=dict(zip(range(0,len(alphas)+1),alphaCols)))
    normals_df = pd.DataFrame(np.array(normals, dtype=object), columns=['normals','centers'], index=alphaCols)
    return {'tetrahedrons':tetras_df, 'triangles': triangles_df, 'edges': edges_df, 'vertices': vertices_df,
            'components': components_df, 'volumes': volumes_df, 'normals' :normals_df, 'crit_val':crit_val, 'crit_index':crit_index }

# alphaShapes_s = alphashape3d.ashape3d(rdfS, alphas)
# alphaShapes_d = alphashape3d.ashape3d(rdfD, alphas)
# alpha_d = getAlphaObjects(rdfD, alphas)


In [ ]:
alpha_s = getAlphaObjects(rdfS, alphas)


In [660]:
alphaShapes_s = alphashape3d.ashape3d(rdfS, alphas)

In [693]:
alphashape3d.surfaceNormals(alphaShapes_s, indexAlpha=15)
#breaks when 11=<a<15

0,normals,[[-0.3433...
1,centers,[[-51.442...


In [715]:
tetras_df = pd.DataFrame(alphaShapes_s[0]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'index4',4:'intervals'},**dict(zip(range(5,len(alphas)+5),alphaCols))})
triangles_df = pd.DataFrame(alphaShapes_s[1]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'on_convex_hull',4:'attached',5:'intervals1',6:'intervals2',7:'intervals3'},**dict(zip(range(8,len(alphas)+8),alphaCols))})
edges_df = pd.DataFrame(alphaShapes_s[2]).rename(columns={**{0:'index1',1:'index2',2:'on_convex_hull',3:'attached',4:'intervals1',5:'intervals2',6:'intervals3'},**dict(zip(range(7,len(alphas)+7),alphaCols))})
vertices_df = pd.DataFrame(alphaShapes_s[3]).rename(columns={**{0:'index',1:'on_convex_hull',2:'intervals1',3:'intervals2'},**dict(zip(range(4,len(alphas)+4),alphaCols))})
vertices_df['x'],vertices_df['y'],vertices_df['z'] = alphaShapes_s[4].T[0],alphaShapes_s[4].T[1],alphaShapes_s[4].T[2]

In [718]:
triangles_df

,index1,index2,index3,on_convex_hull,attached,intervals1,intervals2,intervals3,0_1,0_2,...,81_283,83_176,85_114,87_096,89_125,91_201,93_325,95_499,97_724,100_0
0,1.0,2.0,3.0,0.0,0.0,0.764921,2.086357,4.703001,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,2.0,801.0,0.0,0.0,4.680549,4.703001,8.371330,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,2.0,802.0,0.0,1.0,8.004846,8.189485,8.371330,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,2.0,1751.0,0.0,0.0,2.334550,2.360106,2.369498,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,2.0,1752.0,0.0,0.0,2.077542,2.086357,2.360106,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25730,1485.0,1802.0,1803.0,1.0,1.0,15.020511,65.097545,65.097545,0.0,0.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
25731,1281.0,1804.0,1811.0,1.0,1.0,20.806547,575.229400,575.229400,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25732,1281.0,1803.0,1804.0,1.0,1.0,15.488077,1398.035810,1398.035810,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25733,1733.0,1804.0,1960.0,1.0,1.0,33.106065,260.809626,260.809626,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [719]:
exists = vertices_df[vertices_df['11_22']!= 0.0]
exists

,index,on_convex_hull,intervals1,intervals2,0_1,0_2,0_3,0_4,0_5,0_6,...,87_096,89_125,91_201,93_325,95_499,97_724,100_0,x,y,z
0,1.0,0.0,0.469215,8.750276,3.0,3.0,3.0,3.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,4.160690,-27.631102,-7.857958
1,2.0,0.0,0.469215,8.371330,3.0,3.0,3.0,3.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.241229,-27.590133,-7.674754
2,3.0,0.0,0.574373,5.446330,3.0,3.0,3.0,3.0,3.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.150354,-26.497168,-8.016497
3,4.0,0.0,2.173545,8.789849,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-2.467717,-2.565793,-16.824917
4,5.0,0.0,1.059903,9.593152,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-17.304093,38.892613,-22.964105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,1988.0,0.0,1.282044,7.758876,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,-3.938837,3.530687,-59.688122
1988,1989.0,0.0,0.968185,4.459926,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,7.471219,-19.479819,-66.480688
1989,1990.0,0.0,2.392866,22.306428,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,22.784441,-39.303314,-54.337066
1990,1991.0,0.0,0.088203,40.559168,2.0,2.0,2.0,2.0,2.0,2.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,26.645024,-53.025626,-38.036300


In [705]:
np.any(vertices_df['0_1']==3.0)

True

In [722]:
alphaCols = [str(round(alpha,3)).replace('.','_') for alpha in alphas]
has_isolated_vertices = [alpha.replace('_','.') for alpha in alphaCols if np.any(vertices_df[alpha]==3.0)]
contains_all_points = [alpha.replace('_','.') for alpha in alphaCols if np.all(vertices_df[alpha]<3)]
has_isolated_edges = 

In [741]:
edges_df.loc[(edges_df['81_283']==1.0|2.0)&(edges_df['attached']==0.0)]

TypeError: unsupported operand type(s) for |: 'float' and 'float'

In [302]:
def downsampler(df, species, factor):
    full = df.loc[df.species == species]
    full.loc[:,'uid'] = full.index
    full.loc[:,'animalside'] = full.animal+full.side
    for animalside in full.animalside.unique():
        as_df = full[full.animalside == animalside]
        min_frame, max_frame = as_df.frame.min(), as_df.frame.max()
        candidate_range = np.arange(min_frame, max_frame+1)
        frames_to_pick = list(candidate_range[::10])
        print(frames_to_pick)
        full = full.drop(full[(full['animalside']==animalside)&(full['frame'] not in frames_to_pick)].index)
        print(full)


# SWAP CRX AND CRZ FOR ALL BELOW

In [220]:
#center and superimpose joint alpha shapes
def centerJoints(df, species):
    df = df[df['species']==species[0]].copy()
    RCols = [col for col in df.columns if re.match(r'(c*R[xyz])', col)]
    for col in RCols:
        df[col+'_centered'] = df[col]-df[col].mean()
    return df
    
salCentered = centerJoints(df, 's')
didCentered = centerJoints(df, 'd')

rotationsCentered = getRotationRanges(salCentered, didCentered)
CmaxRx, CmaxcRz, CminRx, CmincRz, CmaxRy, CminRy, CmaxRz, CminRz = rotationsCentered.values()

alpha = 25
alphaSalVert,alphaSalEdge, alphaSalTri = alpha_shape_3D(np.array([salCentered.cRz_centered,salCentered.Ry_centered,salCentered.Rz_centered]).T,alpha)
alphaDidVert,alphaDidEdge, alphaDidTri = alpha_shape_3D(np.array([didCentered.cRz_centered,didCentered.Ry_centered,didCentered.Rz_centered]).T,alpha)


plt.close('all')


fig = plt.figure(figsize=[15,10], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')



format3dPlot(ax0, 'Tegu', (CmincRz, CmaxcRz), (CminRy, CmaxRy), (CminRz, CmaxRz), view3Q)
format3dPlot(ax1, 'Opossum', (CmincRz, CmaxcRz), (CminRy, CmaxRy), (CminRz, CmaxRz), view3Q)

addCosGrid(ax0, (CminRx,CmaxRx), (CminRy,CmaxRy), (CminRz,CmaxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (CminRx,CmaxRx), (CminRy,CmaxRy), (CminRz,CmaxRz), 5, xLevels=1, color='grey', linewidths=0.1)


ax0.plot_trisurf(salCentered.cRz_centered, salCentered.Ry_centered, alphaSalTri, salCentered.Rz_centered, shade=True, color=teguColor, linewidth=0, antialiased=True)
ax1.plot_trisurf(didCentered.cRz_centered, didCentered.Ry_centered, alphaDidTri, didCentered.Rz_centered, shade=True, color=opossumColor, linewidth=0, antialiased=True)

ax0.scatter(salCentered.cRz_centered,salCentered.Ry_centered,salCentered.Rz_centered, s=2, c=orangeYellowSeafoam[6], depthshade=False, edgecolors='None', alpha=0.95)
ax1.scatter(didCentered.cRz_centered,didCentered.Ry_centered,didCentered.Rz_centered, s=2, c=orangeYellowSeafoam[2], depthshade=False, edgecolors='None', alpha=0.95)


# ax0.add_patch(PolygonPatch(hullDXZ, fill=True, color='blue'))






connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()
# plt.savefig('3q.png', dpi=150, transparent=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-150-96946a24d4af>:145: MatplotlibDeprecationWarning: Case-insensitive properties were deprecated in 3.3 and support will be removed two minor releases later
  gridAx.plot_wireframe(xx, yy, czz, rcount = zSize/interval, ccount=ySize/interval, **kwargs)


AttributeError: 'Line3DCollection' object has no property 'zlevels'

In [9]:
scaled_Zma_cols = [col for col in scaled_t_ma_all_data.columns if col[-3:] == 'Zma']
colsForForr = scaled_Zma_cols
# dfSforCorr = dfS.filter(colsForForr)
# dfSforCorr = dfSforCorr.dropna(axis=1, how='all')
# corr = dfSforCorr.corr()
# corr.style.background_gradient(cmap='bwr')
dfDforCorr = dfD.filter(colsForForr)
dfDforCorr_colnames_raw = [col.split('_',1)[1].rsplit('_',1)[0] for col in colsForForr]
dfDforCorr.columns = dfDforCorr_colnames_raw
dfDforCorr = dfDforCorr.dropna(axis=1, how='all')
corr = dfDforCorr.corr()
corr.style.background_gradient(cmap='bwr')

,lat_cdl,lat_crn,pec_crn,pec_int,pec_cdl,trl,dsc,dac,bic,tmj
lat_cdl,1.000000,0.778650,-0.046383,0.248244,0.305269,0.478560,-0.451596,-0.175176,0.446266,0.743981
lat_crn,0.778650,1.000000,-0.113531,0.169180,0.261982,0.495467,-0.456976,-0.340188,0.161647,0.652616
pec_crn,-0.046383,-0.113531,1.000000,0.753182,0.545561,-0.460594,0.055906,-0.464617,0.322532,-0.111656
pec_int,0.248244,0.169180,0.753182,1.000000,0.928390,-0.261588,-0.315568,-0.767559,0.575061,0.078853
pec_cdl,0.305269,0.261982,0.545561,0.928390,1.000000,0.061553,-0.307862,-0.762522,0.551862,0.087666
trl,0.478560,0.495467,-0.460594,-0.261588,0.061553,1.000000,0.291802,0.396915,0.141848,nan
dsc,-0.451596,-0.456976,0.055906,-0.315568,-0.307862,0.291802,1.000000,0.542833,-0.206050,-0.269684
dac,-0.175176,-0.340188,-0.464617,-0.767559,-0.762522,0.396915,0.542833,1.000000,-0.408894,0.310535
bic,0.446266,0.161647,0.322532,0.575061,0.551862,0.141848,-0.206050,-0.408894,1.000000,0.079723
tmj,0.743981,0.652616,-0.111656,0.078853,0.087666,nan,-0.269684,0.310535,0.079723,1.000000


In [218]:
from scipy.spatial import Delaunay
import numpy as np
from collections import defaultdict

def alpha_shape_3D(pos, alpha):
    """
    Compute the alpha shape (concave hull) of a set of 3D points.
    Parameters:
        pos - np.array of shape (n,3) points.
        alpha - alpha value.
    return
        outer surface vertex indices, edge indices, and triangle indices
    """

    tetra = Delaunay(pos)
    # Find radius of the circumsphere.
    # By definition, radius of the sphere fitting inside the tetrahedral needs 
    # to be smaller than alpha value
    # http://mathworld.wolfram.com/Circumsphere.html
    tetrapos = np.take(pos,tetra.vertices,axis=0)
    normsq = np.sum(tetrapos**2,axis=2)[:,:,None]
    ones = np.ones((tetrapos.shape[0],tetrapos.shape[1],1))
    a = np.linalg.det(np.concatenate((tetrapos,ones),axis=2))
    Dx = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[1,2]],ones),axis=2))
    Dy = -np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,2]],ones),axis=2))
    Dz = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,1]],ones),axis=2))
    c = np.linalg.det(np.concatenate((normsq,tetrapos),axis=2))
    r = np.sqrt(Dx**2+Dy**2+Dz**2-4*a*c)/(2*np.abs(a))
    # Find tetrahedrals
    tetras = tetra.vertices[r<alpha,:]
    # triangles
    TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
    Triangles = tetras[:,TriComb].reshape(-1,3)
    Triangles = np.sort(Triangles,axis=1)

    # Remove triangles that occurs twice, because they are within shapes
    TrianglesDict = defaultdict(int)
    for tri in Triangles:TrianglesDict[tuple(tri)] += 1
    Triangles=np.array([tri for tri in TrianglesDict if TrianglesDict[tri] ==1])
    #edges
    EdgeComb=np.array([(0, 1), (0, 2), (1, 2)])
    Edges=Triangles[:,EdgeComb].reshape(-1,2)
    Edges=np.sort(Edges,axis=1)
    Edges=np.unique(Edges,axis=0)

    Vertices = np.unique(Edges)
    return Vertices,Edges,Triangles

In [51]:
dfD[(dfD['Rx']>0)&(dfD['Rx']<5)&(dfD['Ry']>45)&(dfD['Ry']<50)][['scaled_dac_Xma','scaled_dac_Yma','scaled_dac_Zma']]

,scaled_dac_Xma,scaled_dac_Yma,scaled_dac_Zma
3691,-0.011453,0.009194,-0.005271
5168,-0.011008,0.010386,-0.005277


In [53]:
#color by momentarm

def RgbMa(df, species, muscleName, xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.filter(like=muscleName)
    df = df.replace(np.NaN, 0)
    Xma = df.filter(like='Xma')
    Yma = df.filter(like='Yma')
    Zma = df.filter(like='Zma')
    df['posXma'] = np.where(Xma > 0, Xma, 0)
    df['posYma'] = np.where(Yma > 0, Yma, 0)
    df['posZma'] = np.where(Zma > 0, Zma, 0)
    df['negXma'] = np.where(Xma < 0, abs(Xma), 0)
    df['negYma'] = np.where(Yma < 0, abs(Yma), 0)
    df['negZma'] = np.where(Zma < 0, abs(Zma), 0)
    df=df.drop([Xma.columns[0],Yma.columns[0],Zma.columns[0]], axis=1)
    mmMax = df.max().max()
    df=df/mmMax
    df['Xma'] = df['posXma'] if xPos else df['negXma']
    df['Yma'] = df['posYma'] if yPos else df['negYma']
    df['Zma'] = df['posZma'] if zPos else df['negZma']
    totalXYZ = df[['Xma','Yma','Zma']].sum(axis=1)
    df['scale'] = preprocessing.MinMaxScaler().fit_transform(totalXYZ.values.reshape(-1, 1))
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species[0]]
    return(df[['Xma','Yma','Zma','scale']])



# stanceS = RgbMa(df, 'salvator', 'scaled_dac', xPos=True, yPos=False, zPos=False, normalizeAcrossSpecies=True)
# stanceD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=True, yPos=False, zPos=False, normalizeAcrossSpecies=True)
# swingS = RgbMa(df, 'salvator', 'scaled_dac', xPos=False, yPos=True, zPos=True, normalizeAcrossSpecies=True)
# swingD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=False, yPos=True, zPos=True, normalizeAcrossSpecies=True)

stanceS = RgbMa(df, 'salvator', 'scaled_dac', xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True)
stanceD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True)
swingS = RgbMa(df, 'salvator', 'scaled_dac', xPos=False, yPos=False, zPos=False, normalizeAcrossSpecies=True)
swingD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=False, yPos=False, zPos=False, normalizeAcrossSpecies=True)

plt.close('all')


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(221, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(222, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(223, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(224, projection='3d', proj_type = 'ortho')


# 'Tegu Stance R: Prn, G: Add, B: Ext'
# 'Opossum Stance R: Prn, G: Add, B: Ext'

format3dPlot(ax0, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)
format3dPlot(ax1, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)
format3dPlot(ax2, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)
format3dPlot(ax3, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=stanceS['scale']*15, c=stanceS[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=stanceD['scale']*15, c=stanceD[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=swingS['scale']*15, c=swingS[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=swingD['scale']*15, c=swingD[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection4 = fig.canvas.mpl_connect('motion_notify_event', on_move4)

plt.show()

#threshold, then pca?

# plt.savefig('dsc.png', dpi=150, transparent=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:

def divergingMa(df, species, muscleName, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.filter(like=muscleName)
    df = df.replace(np.NaN, 0)
    mmMax = df.max().max()
    mmAbsMax = abs(df).max().max()
    mmMin = df.min().min()
    mmAbsMin = abs(df).min().min()
    MaxMinMax = max((abs(mmMax),abs(mmMin)))
    def MinMaxScalerPrescribedRange(X, min_input, max_input, min_output=0, max_output=1):
        X_std = (X - min_input) / (max_input - min_input)
        X_scaled = X_std * (max_output - min_output) + min_output
        return X_scaled
    Xma = df.filter(like='Xma')/MaxMinMax   #maxabs scale across all 3 axes
    Yma = df.filter(like='Yma')/MaxMinMax
    Zma = df.filter(like='Zma')/MaxMinMax
    df['Xmag'] = abs(Xma)   #magnitude of moment arm
    df['Ymag'] = abs(Yma)
    df['Zmag'] = abs(Zma)
    df['Xscale'] = MinMaxScalerPrescribedRange(Xma, -1, 1)    #get color mapping var by normalizing moment arms to lie between 0 and 1 (with -1 and 1 as min and max input values)
    df['Yscale'] = MinMaxScalerPrescribedRange(Yma, -1, 1)
    df['Zscale'] = MinMaxScalerPrescribedRange(Zma, -1, 1)
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species[0]]
    return(df)



pec_crn_S = divergingMa(df, 'salvator', 'scaled_dac', normalizeAcrossSpecies=True)
pec_crn_D = divergingMa(df, 'didelphis', 'scaled_dac', normalizeAcrossSpecies=True)

plt.close('all')


fig = plt.figure(figsize=[12,18], constrained_layout=True)
ax0 = fig.add_subplot(321, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(322, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(323, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(324, projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(325, projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(326, projection='3d', proj_type = 'ortho')


# 'Tegu Stance R: Prn, G: Add, B: Ext'
# 'Opossum Stance R: Prn, G: Add, B: Ext'

format3dPlot(ax0, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax1, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax2, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax3, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax4, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax5, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax4, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax5, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Xmag']*30, c=pec_crn_S['Xscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Xmag']*30, c=pec_crn_D['Xscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Ymag']*30, c=pec_crn_S['Yscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Ymag']*30, c=pec_crn_D['Yscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax4.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Zmag']*30, c=pec_crn_S['Zscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax5.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Zmag']*30, c=pec_crn_D['Zscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection6 = fig.canvas.mpl_connect('motion_notify_event', on_move6)

plt.show()

# plt.savefig('dac_div_3Q.png', dpi=150, transparent=True)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
#color by momentarm
dfRgb = df.copy()
dfSrgb = df[df['species']=='s']
dfDrgb = df[df['species']=='d']

scaled_muscle_cols = [col for col in dfRgb.columns if col.split('_')[0] == 'scaled' and col[-2:] == 'ma']
dfRgb[scaled_muscle_cols] = dfRgb[scaled_muscle_cols].replace(np.NaN, 0)

lat_crn = dfRgb[scaled_muscle_cols].iloc[:, 3:6]
latmax = lat_crn.max().max()
latmin = lat_crn.min().min()

latNewX = lat_crn.scaled_lat_crn_Xma+abs(latmin)
latNewY = lat_crn.scaled_lat_crn_Yma+abs(latmin)
latNewZ = lat_crn.scaled_lat_crn_Zma+abs(latmin)

latNewXnorm = latNewX/(latmax+abs(latmin))
latNewYnorm = latNewY/(latmax+abs(latmin))
latNewZnorm = latNewZ/(latmax+abs(latmin))



# scaler = sklearn.preprocessing.MinMaxScaler().fit(dfRgb[scaled_muscle_cols])
# dfRgb[scaled_muscle_cols] = scaler.transform(dfRgb[scaled_muscle_cols])
# DrgbXYZ = scaler.transform(dfDrgb[['scaled_Tx','scaled_Ty','scaled_Tz']])

# # SrgbXYZ = np.array([SrgbX, SrgbY, SrgbZ]).reshape(3,len(SrgbX)).T
# # DrgbXYZ = np.array([DrgbX, DrgbY, DrgbZ]).reshape(3,len(DrgbX)).T
plt.close('all')


fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)
format3dPlot(ax1, 'Opossum', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=2, c='black', depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=2, c='black', depthshade=False, edgecolors='None')
ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=2, c=np.array([np.zeros(len(latNewXnorm)),latNewYnorm,np.zeros(len(latNewXnorm)),latNewXnorm]).T[dfSrgb.index], depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=2, c=np.array([np.zeros(len(latNewXnorm)),latNewYnorm,np.zeros(len(latNewXnorm)),latNewXnorm]).T[dfDrgb.index], depthshade=False, edgecolors='None')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [132]:
 dfRgb[scaled_muscle_cols]

,scaled_lat_cdl_Xma,scaled_lat_cdl_Yma,scaled_lat_cdl_Zma,scaled_lat_crn_Xma,scaled_lat_crn_Yma,scaled_lat_crn_Zma,scaled_pec_crn_Xma,scaled_pec_crn_Yma,scaled_pec_crn_Zma,scaled_pec_int_Xma,...,scaled_dsc_Zma,scaled_dac_Xma,scaled_dac_Yma,scaled_dac_Zma,scaled_tmj_Xma,scaled_tmj_Yma,scaled_tmj_Zma,scaled_bic_Xma,scaled_bic_Yma,scaled_bic_Zma
0,0.012859,0.006960,-0.000229,0.022645,0.010314,-0.003437,-0.006570,-0.002715,-0.010182,-0.009046,...,-0.013349,-0.001171,0.017018,0.000238,0.0,0.0,0.0,0.000000,0.000000,0.000000
1,0.012505,0.007474,0.000368,0.021933,0.010731,-0.003034,-0.005310,-0.004207,-0.009598,-0.006805,...,-0.013153,-0.001645,0.017111,0.000644,0.0,0.0,0.0,0.000000,0.000000,0.000000
2,0.011591,0.007343,0.001136,0.020691,0.010317,-0.002525,-0.001301,-0.007083,-0.007018,0.000024,...,-0.012470,-0.004141,0.016628,0.000210,0.0,0.0,0.0,0.000000,0.000000,0.000000
3,0.011346,0.006381,-0.005551,0.019934,0.007633,-0.014985,0.003120,-0.011373,-0.004881,0.006568,...,-0.009181,-0.005483,0.014815,0.001495,0.0,0.0,0.0,0.000000,0.000000,0.000000
4,0.008757,0.004531,-0.006397,0.017222,0.006807,-0.016707,0.008865,-0.013984,-0.002230,0.015560,...,-0.008108,-0.007360,0.011620,0.001595,0.0,0.0,0.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5028,0.000000,0.000000,0.000000,0.012402,-0.022690,-0.002173,-0.000514,0.011865,0.008826,0.000000,...,-0.003891,-0.006769,0.008313,-0.000005,0.0,0.0,0.0,-0.007131,0.008228,0.012621
5029,0.000000,0.000000,0.000000,0.015883,-0.016479,0.003090,-0.001225,0.016797,0.012043,0.000000,...,0.000707,-0.008765,0.009636,0.002292,0.0,0.0,0.0,-0.012104,0.011546,0.011563
5030,0.000000,0.000000,0.000000,0.016061,-0.013051,0.009394,-0.001191,0.019642,0.013506,0.000000,...,0.003928,-0.010812,0.011501,0.003386,0.0,0.0,0.0,-0.012179,0.010731,0.008750
5031,0.000000,0.000000,0.000000,0.016148,-0.012392,0.011521,-0.000220,0.019816,0.013365,0.000000,...,0.004584,-0.012184,0.011221,0.003200,0.0,0.0,0.0,-0.007536,0.007560,0.006150


In [185]:
#color by translation
dfRgb = df.copy()
# dfSrgb = df[df['species']=='s']
# dfDrgb = df[df['species']=='d']

scaled_Ts = dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']]
tMin = scaled_Ts.min().min()
tMax = scaled_Ts.max().max()

scalerMinMax = preprocessing.MinMaxScaler((tMin,tMax)).fit(dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']])
dfRgbMinMax = scalerMinMax.transform(dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']])
scaler01 = preprocessing.MinMaxScaler((0,1)).fit(dfRgbMinMax)
dfRgb01 = scaler01.transform(dfRgbMinMax)

dfSrgb = dfRgb01[dfRgb[dfRgb['species']=='s'].index]
dfDrgb = dfRgb01[dfRgb[dfRgb['species']=='d'].index]


plt.close('all')


fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)
format3dPlot(ax1, 'Opossum', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=3, c=dfSrgb, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=3, c=dfDrgb, depthshade=False, edgecolors='None')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:
sklearn.preprocessing

AttributeError: module 'sklearn' has no attribute 'preprocessing'

In [320]:
#PCA
from sklearn.decomposition import PCA

XYZ = df[['cRx','Ry','Rz']].to_numpy()
XYZ = sklearn.preprocessing.normalize(XYZ)
aPCA = PCA().fit(XYZ)

sXYZ = dfS[['cRx','Ry','Rz']].to_numpy()
sXYZ = sklearn.preprocessing.normalize(sXYZ)
sPCA = PCA().fit(sXYZ)

dXYZ = dfD[['cRx','Ry','Rz']].to_numpy()
dXYZ = sklearn.preprocessing.normalize(dXYZ)
dPCA = PCA().fit(dXYZ)


aPCAs = aPCA.transform(sXYZ).T
aPCAd = aPCA.transform(dXYZ).T
sPCAs = sPCA.transform(sXYZ).T
sPCAd = sPCA.transform(dXYZ).T
dPCAs = dPCA.transform(sXYZ).T
dPCAd = dPCA.transform(dXYZ).T

maxPC1a = max([max(aPCAs[0]),max(aPCAd[0])])
minPC1a = min([min(aPCAs[0]),min(aPCAd[0])])
maxPC2a = max([max(aPCAs[1]),max(aPCAd[1])])
minPC2a = min([min(aPCAs[1]),min(aPCAd[1])])
maxPC3a = max([max(aPCAs[2]),max(aPCAd[2])])
minPC3a = min([min(aPCAs[2]),min(aPCAd[2])])

maxPC1s = max([max(sPCAs[0]),max(sPCAd[0])])
minPC1s = min([min(sPCAs[0]),min(sPCAd[0])])
maxPC2s = max([max(sPCAs[1]),max(sPCAd[1])])
minPC2s = min([min(sPCAs[1]),min(sPCAd[1])])
maxPC3s = max([max(sPCAs[2]),max(sPCAd[2])])
minPC3s = min([min(sPCAs[2]),min(sPCAd[2])])

maxPC1d = max([max(dPCAs[0]),max(dPCAd[0])])
minPC1d = min([min(dPCAs[0]),min(dPCAd[0])])
maxPC2d = max([max(dPCAs[1]),max(dPCAd[1])])
minPC2d = min([min(dPCAs[1]),min(dPCAd[1])])
maxPC3d = max([max(dPCAs[2]),max(dPCAd[2])])
minPC3d = min([min(dPCAs[2]),min(dPCAd[2])])

plt.close('all')



fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0PCA = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1PCA = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0PCA, 'Tegu', (minPC1a, maxPC1a), (minPC2a, maxPC2a), (minPC3a, maxPC3a), viewXY, mode='pca')
format3dPlot(ax1PCA, 'Opossum', (minPC1a, maxPC1a), (minPC2a, maxPC2a), (minPC3a, maxPC3a), viewXY, mode='pca')

ax0PCA.scatter(aPCAs[0],aPCAs[1],aPCAs[2], s=3, depthshade=True)
ax1PCA.scatter(aPCAd[0],aPCAd[1],aPCAd[2], s=3, depthshade=True)



plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
def alpha_shape(points, alpha, only_outer=True): #c/o "Harold" on stackoverflow
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: np.array of shape (n, 3) points.
    :param alpha: alpha value.
    :param only_outer: boolean value to specify if we keep only the outer border
    or also inner edges.
    :return: set of (i,j) pairs representing edges of the alpha-shape. (i,j) are
    the indices in the points array.
    """
    assert points.shape[0] > 3, "Need at least four points"

    def add_edge(edges, i, j):
        """
        Add a line between the i-th and j-th points,
        if not in the set already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            if only_outer:
                # if both neighboring triangles are in shape, it's not a boundary edge
                if (j, i) in edges:
                    edges.remove((j, i))
            return
        edges.add((i, j))

    tri = Delaunay(points)
    edges = set()
    # Loop over triangles:
    # ia, ib, ic, id = indices of corner points of the tetrahedron
    print(tri.vertices.shape)
    for ia, ib, ic, id in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        pd = points[id]

        # Computing radius of tetrahedron Circumsphere
        # http://mathworld.wolfram.com/Circumsphere.html

        pa2 = np.dot(pa, pa)
        pb2 = np.dot(pb, pb)
        pc2 = np.dot(pc, pc)
        pd2 = np.dot(pd, pd)

        a = np.linalg.det(np.array([np.append(pa, 1), np.append(pb, 1), np.append(pc, 1), np.append(pd, 1)]))

        Dx = np.linalg.det(np.array([np.array([pa2, pa[1], pa[2], 1]),
                                     np.array([pb2, pb[1], pb[2], 1]),
                                     np.array([pc2, pc[1], pc[2], 1]),
                                     np.array([pd2, pd[1], pd[2], 1])]))

        Dy = - np.linalg.det(np.array([np.array([pa2, pa[0], pa[2], 1]),
                                       np.array([pb2, pb[0], pb[2], 1]),
                                       np.array([pc2, pc[0], pc[2], 1]),
                                       np.array([pd2, pd[0], pd[2], 1])]))

        Dz = np.linalg.det(np.array([np.array([pa2, pa[0], pa[1], 1]),
                                     np.array([pb2, pb[0], pb[1], 1]),
                                     np.array([pc2, pc[0], pc[1], 1]),
                                     np.array([pd2, pd[0], pd[1], 1])]))

        c = np.linalg.det(np.array([np.array([pa2, pa[0], pa[1], pa[2]]),
                                    np.array([pb2, pb[0], pb[1], pb[2]]),
                                    np.array([pc2, pc[0], pc[1], pc[2]]),
                                    np.array([pd2, pd[0], pd[1], pd[2]])]))

        circum_r = math.sqrt(math.pow(Dx, 2) + math.pow(Dy, 2) + math.pow(Dz, 2) - 4 * a * c) / (2 * abs(a))
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, id)
            add_edge(edges, id, ia)
            add_edge(edges, ia, ic)
            add_edge(edges, ib, id)
    return edges